## Fibonacci 

In the following example, the tasking concept is used to compute Fibonacci numbers recursively.

This is the recursive approach to compute Fibonacci without using OpenMP. 

In [ ]:
int fib(int n) {
    if (n < 2) return n;
    int x = fib(n - 1);
    int y = fib(n - 2);
    return x+y;
}

In [ ]:
int n = 8;
printf ("fib(%d) = %d\n", n, fib(n));

Now let's look at the same example done with tasking. 

* Only one thread executes single, we only need to print once. 
* For every fib() call two tasks are generated, one computes fib(n-1), one fib(n-2). 
* Tasks will be recursively generated until the argument passed to fib() is less than 2. 
* Taskwait ensures that the two tasks generated are completed before we return the sum. 

In [ ]:
#pragma cling load("libomp.so")
#include <stdlib.h>
#include <stdio.h>
#include <omp.h>

int fib(int n) {
    if (n < 2) return n;
    int x, y;
    #pragma omp task shared(x) firstprivate(n)
    {
        x = fib(n - 1);
    }
    #pragma omp task shared(y) firstprivate(n)
    {
        y = fib(n - 2);
    }
    #pragma omp taskwait
    return x+y;
}

In [ ]:
int n_threads = 4;
omp_set_num_threads(n_threads);

int n = 8;
#pragma omp parallel shared(n)
{
    #pragma omp single
    printf ("fib(%d) = %d\n", n, fib(n));
}